实例化 Embedding 层

In [1]:
from keras.layers import Embedding

# Embedding 层至少需要两个参数：
# 标记的个数（这里是 1000），即最大单词索引+1
# 嵌入的维度，这里是 64
embedding_layer = Embedding(1000, 64)

可以将 Embedding 层理解为字段，将整数索引（表示特定单词）映射为密集向量。它接收整数作为输入，并在内部字典中查找这些整数，然后返回相关联的向量。

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 10000  # 作为特征的单词个数
max_len = 20  # 在这么多单词后阶段文本（这些单词都属于 max_features 中最常见的单词）

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)  # 将数据加载为正数列表
x_train = sequence.pad_sequences(x_train, maxlen=max_len)  # 将整数列表转换为 (samples, maxlen) 的二维整数张量
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 10000 是词汇大小，8 是输出嵌入层维度
# input_length 是输入序列长度
# 输出为 (batch_size, input_length, output_dim)，即 (samples, max_len, 8)
model.add(Embedding(10000, 8, input_length=max_len))
model.add(Flatten())  # 将三维的嵌入张量展平为（samples, max_len*8）的二维张量
model.add(Dense(1, activation='sigmoid'))  # 用于分类
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [4]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 4s 4ms/step - loss: 0.6646 - acc: 0.6370 - val_loss: 0.6104 - val_acc: 0.7032
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5354 - acc: 0.7510 - val_loss: 0.5214 - val_acc: 0.7334
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4591 - acc: 0.7867 - val_loss: 0.4983 - val_acc: 0.7454
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4210 - acc: 0.8090 - val_loss: 0.4906 - val_acc: 0.7522
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3940 - acc: 0.8220 - val_loss: 0.4910 - val_acc: 0.7550
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3721 - acc: 0.8348 - val_loss: 0.4926 - val_acc: 0.7592
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.3519 - acc: 0.8451 - val_loss: 0.4994 - val_acc: 0.7558
Epoch 8/10
625/625 [==============================] - 2s 3ms/step - loss: 0.

得到的精度约 96%，考虑到仅查看每条评论的前 20 个单词，这个结果过还是相当不错的。

另外要注意，仅仅将嵌入序列展开并在上面训练一个 Dense 层，会导致模型对输入序列中的每个单词单独处理，而没有考虑单词之间的关系和句子结构。

更好的做法是在嵌入序列上添加循环层或一维卷积层，将每个序列作为整体来学习特征。